In [42]:
import os
import pandas as pd
from pathlib import Path


In [43]:
OUT_PATH = os.path.join('../../../', 'data', 'processed', 'areas')

raw_data = pd.read_csv('../../../data/raw/education/2223_sl_pcon_data_provisional.csv', usecols= {
        'time_period',
        'geographic_level',
        'pcon_code', 
        'pcon_name', 
        # 'version', 
        'avg_att8', 
        'avg_p8score', 
        'pt_l2basics_94',
        'pt_l2basics_95'
    })

raw_data

,time_period,geographic_level,pcon_code,pcon_name,avg_att8,pt_l2basics_95,pt_l2basics_94,avg_p8score
0,202223,National,NaN,NaN,46.3,45.3,65.1,-0.03
1,202223,Parliamentary constituency,E14000530,Aldershot,40.9,35.8,56.5,-0.63
2,202223,Parliamentary constituency,E14000531,Aldridge-Brownhills,42.3,35.3,58.0,-0.36
3,202223,Parliamentary constituency,E14000532,Altrincham and Sale West,59.5,67.4,82.9,0.40
4,202223,Parliamentary constituency,E14000533,Amber Valley,43.4,38.4,60.0,-0.15
...,...,...,...,...,...,...,...,...
529,202223,Parliamentary constituency,E14001058,Wyre Forest,41.8,38.5,56.4,-0.27
530,202223,Parliamentary constituency,E14001059,Wythenshawe and Sale East,44.4,41.6,57.8,-0.42
531,202223,Parliamentary constituency,E14001060,Yeovil,43.0,39.0,60.5,-0.28
532,202223,Parliamentary constituency,E14001061,York Central,48.6,46.8,68.0,0.09


In [44]:
filtered = raw_data[(raw_data['time_period']==202223) & (raw_data['geographic_level']=='Parliamentary constituency')]
attainment_scores_pcon_2010 = (
    filtered.drop(columns='geographic_level')
    .pivot_table(index=['pcon_code', 'pcon_name'], values = ['avg_p8score', 'avg_att8','pt_l2basics_94', 'pt_l2basics_95'])
).to_csv(os.path.join(OUT_PATH, 'attainment_scores_pcon_2010.csv'))